## Домашнее задание<a class='anchor' id='hw'>

In [4]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Task 1. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

In [3]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [17]:
def PCA(X,n):
    
    #масштабирование данных
    X = X.astype(float)
    X = standard_scale(X)
    
    # Найдем собственные векторы и собственные значения 
    covariance_matrix = X.T @ X
    eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

    # сформируем список кортежей (собственное значение, собственный вектор)
    eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

    # и отсортируем список по убыванию собственных значений
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
    W = np.hstack([eig_pairs[i][1].reshape(4,1) for i in range(n)])
    
    # Сформируем новую матрицу "объекты-признаки"
    Z = X.dot(W)
    
    return Z 

In [34]:
# Загрузим датасет и разобьём на выборки 
iris = datasets.load_iris()
X = iris.data
Y=iris.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=66)

In [39]:
# в качестве модели выберем DecisionTree
from sklearn.tree import DecisionTreeClassifier

clas_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, 
                             min_samples_leaf=3).fit(X_train, y_train)
clas_tree.score(X_test, y_test)

0.9333333333333333

#### Теперь обучим модель на выборке, предварительно подвергнув её PCA

In [42]:
X_train_decomp = PCA(X_train,2)
X_test_decomp = PCA(X_test,2)
clas_tree = DecisionTreeClassifier(criterion='entropy', max_depth=5, 
                             min_samples_leaf=3).fit(X_train_decomp, y_train)
clas_tree.score(X_test_decomp, y_test)

0.8444444444444444

________

Как результат, точность предсказаний выше у модели без PCA (93% против 84%)